# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [44]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import pandas as pd

In [45]:
wine_dataset = load_wine()
wine_dataframe = pd.DataFrame(wine_dataset['data'], columns=wine_dataset.feature_names)

In [46]:
wine_dataframe.columns

Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline'],
      dtype='object')

In [47]:
selected_cols = [wine_dataframe.columns[i] for i in (9, 11, 12)]
selected_cols

['color_intensity', 'od280/od315_of_diluted_wines', 'proline']

In [48]:
X = wine_dataframe[selected_cols]
y = wine_dataset['target']

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=17)

**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [49]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [50]:
skf = StratifiedKFold(n_splits=5, random_state=17, shuffle=True)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [52]:
lr = LogisticRegression(random_state=17, multi_class='ovr')
scores = cross_val_score(lr, X, y, cv=skf)
print('Средняя доля правильных ответов кросс-валидации', scores.mean())

Средняя доля правильных ответов кросс-валидации 0.9330158730158731


In [53]:
lr.fit(x_train, y_train)
test_score = lr.score(x_test, y_test)
print("Доля правильных ответов на тестовой выборке:", test_score)

Доля правильных ответов на тестовой выборке: 1.0


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [54]:
from sklearn.svm import SVC

In [55]:
svс = SVC(random_state=17)
scores = cross_val_score(svс, X, y, cv=skf)
print('Средняя доля правильных ответов кросс-валидации', scores.mean())

Средняя доля правильных ответов кросс-валидации 0.6803174603174603


In [56]:
svс.fit(x_train, y_train)
test_score = svс.score(x_test, y_test)
print("Доля правильных ответов на тестовой выборке:", test_score)

Доля правильных ответов на тестовой выборке: 0.6666666666666666


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [57]:
confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0])

array([[0, 2],
       [1, 1]])

In [58]:
from sklearn.metrics import classification_report, confusion_matrix

In [59]:
lr_pred = lr.predict(x_test)
lr_cm = confusion_matrix(y_test, lr_pred)
print(lr_cm)

[[ 8  0  0]
 [ 0 16  0]
 [ 0  0 12]]


*Модель ни разу не ошиблась*

In [60]:
svc_pred = svс.predict(x_test)
svc_cm = confusion_matrix(y_test, svc_pred)
print(svc_cm)

[[ 8  0  0]
 [ 0 16  0]
 [ 1 11  0]]


*Модель чаще всего путает класс 2 с классом 1*

### 5.
Для каждой модели выведите classification report.

In [63]:
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00        12

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



In [64]:
print(classification_report(y_test, svc_pred))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.59      1.00      0.74        16
           2       0.00      0.00      0.00        12

    accuracy                           0.67        36
   macro avg       0.49      0.67      0.56        36
weighted avg       0.46      0.67      0.54        36



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
